# Signal Chain Reconstruction

In [ ]:
import torch
from torch import nn
import torchaudio
import os
import numpy as np
import matplotlib.pyplot as plt

### Load dataset

In [ ]:
from src.gtfxdataset import GtFxDataset

AUDIO_DIR = "_assets/DATASET/GT-FX-C53/"
EVU_ANNOTATIONS_FILE = os.path.join(AUDIO_DIR, "evaluation.csv")

SAMPLE_RATE = 22050
NUM_SAMPLES = 22050*3

EFFECT_MAP = ["distortion", "chorus", "tremolo", "delay", "reverb"]


if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
print(f"Using device {device}")

mfcc = torchaudio.transforms.MFCC(
    sample_rate = SAMPLE_RATE, 
    n_mfcc = 64,
    melkwargs = {
        "n_fft": 1024,
        "hop_length": 1024,
        "n_mels": 64,
        "center": False})

fxData = GtFxDataset(EVU_ANNOTATIONS_FILE,
                        AUDIO_DIR,
                        mfcc,
                        SAMPLE_RATE,
                        NUM_SAMPLES,
                        device)
    

#### Reconstruct Singal Chain (singal)

In [ ]:
from src.classifier import model as cModel
from src.classifier import train as cTrain
from src.extrector import model as eModel
from src.extrector import train as eTrain
from src.effectapplier import EffectApplier
from src.util import play_audio, similarity_percentage
import pandas as pd
import random

_WEIGHTS_DIR = "_weights/Legacy/"
_DRY_SAMPLE_DIR = "_assets/DATASET/GT-FX-DRY"

CLASSIFY_WEIGHTS = os.path.join(_WEIGHTS_DIR, "c53_classify_15.pth")

index = random.randint(0, len(fxData))
test_data = fxData[index]

sample_file = fxData.get_audio_sample_filename(index) + ".wav"
dry_file =  "C50-" + sample_file[4:8] + ".wav"

classify = cModel.Classifier().to(device)
classify.load_state_dict(torch.load(CLASSIFY_WEIGHTS))

log = cTrain.test_single(classify, test_data)

extract = eModel.Extractor().to(device)
eff = EffectApplier(_DRY_SAMPLE_DIR, dry_file)

table = []
para_list = ['Gain', 'Depth', 'Rate', 'Delay Time', 'Decay']
for fx, state in enumerate(log):
    table.append([EFFECT_MAP[fx],
                 "State",
                 "Activate" if test_data[1][fx] > 0.0 else "Bypass",
                 "Activate" if state == 1 else "Bypass"])

    if state == 1:
        WEIGHTS_FILE = os.path.join(_WEIGHTS_DIR, "c53_parameter_" + str(fx) + "_20.pth")
        extract.load_state_dict(torch.load(WEIGHTS_FILE))
        val = eTrain.test_single(extract, test_data, fx)
        eff.addEffect(fx, val, mode="manual")
    else:
        val = -1.0

    table.append([EFFECT_MAP[fx],
                para_list[fx],
                "N/A" if test_data[1][fx] < 1 else round(test_data[3][fx].item(), 2),
                "N/A" if val < 0 else val])
remix = eff.generate()
waveform, sr = torchaudio.load(os.path.join(AUDIO_DIR, sample_file))

print(f"Test file: {sample_file}")
play_audio(waveform, sr)

print(f"Remix by CNN Model:")
play_audio(remix, sr)

resampler = torchaudio.transforms.Resample(44100, 22050)

waveform = resampler(waveform)
waveform = mfcc(waveform).reshape(64, 64)
remix = resampler(remix)
remix = mfcc(remix).reshape(64, 64)

similarity = similarity_percentage(waveform, remix)

print(f"Similarity Percentage: {similarity:.2f}%")

df = pd.DataFrame(table)
df.columns = ["Effect", "Item", "Expected", "Predicted"]
df2 = df.groupby(['Effect', "Item"])
df2.first()


#### Evaluate Signal Chain Reconstruction

In [ ]:
from src.classifier import model as cModel
from src.classifier import train as cTrain
from src.extrector import model as eModel
from src.extrector import train as eTrain
from src.effectapplier import EffectApplier
from src.util import similarity_percentage
import pandas as pd


EXPERIMENT_NAME = "c53_signal_chain"
EVU_DIR = "_log/Legacy/Evaluation/"

_WEIGHTS_DIR = "_weights/Legacy/"
_DRY_SAMPLE_DIR = "_assets/DATASET/GT-FX-DRY"
CLASSIFY_WEIGHTS = os.path.join(_WEIGHTS_DIR, "c53_classify_15.pth")

if not os.path.exists('%s' % EVU_DIR):
    os.makedirs('%s' % EVU_DIR)

table = []

for index in range(len(fxData)):  
    test_data = fxData[index]

    sample_file = fxData.get_audio_sample_filename(index) + ".wav"
    dry_file =  "C50-" + sample_file[4:8] + ".wav"

    classify = cModel.Classifier().to(device)
    classify.load_state_dict(torch.load(CLASSIFY_WEIGHTS))

    classify_log = cTrain.test_single(classify, test_data)

    extract = eModel.Extractor().to(device)
    eff = EffectApplier(_DRY_SAMPLE_DIR, dry_file)


    log = [sample_file[:-4]]
    for fx, pred_state in enumerate(classify_log):
        
        state = test_data[1][fx]
        log.append(state * pred_state)

        if state > 0:
            WEIGHTS_FILE = os.path.join(_WEIGHTS_DIR, "c53_parameter_" + str(fx) + "_20.pth")
            extract.load_state_dict(torch.load(WEIGHTS_FILE))
            pred_val = eTrain.test_single(extract, test_data, fx)
            eff.addEffect(fx, val, mode="manual")
            val = round(test_data[3][fx].item(), 2)
            error = round(pred_val - val, 2)
        else:
            error = -1.0
        log.append(error)

    remix = eff.generate()
    waveform, _ = torchaudio.load(os.path.join(AUDIO_DIR, sample_file))
    resampler = torchaudio.transforms.Resample(44100, 22050)

    waveform = resampler(waveform)
    waveform = mfcc(waveform).reshape(64, 64)
    remix = resampler(remix)
    remix = mfcc(remix).reshape(64, 64)
    similarity = similarity_percentage(waveform, remix)

    log.append(round(similarity, 1))
    table.append(log)
    

arr = np.array(table)
np.save(EVU_DIR + EXPERIMENT_NAME + "_evaluation.npy", arr)